In [4]:

import tensorflow as tf
from tensorflow import keras
import numpy as np
import json
from sklearn.model_selection import train_test_split
import os
import  random
from tensorflow.keras import layers
import keras as keras
from keras.applications import  Xception
from PIL import Image
import random
print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.config.threading.set_intra_op_parallelism_threads(4)
tf.config.threading.set_inter_op_parallelism_threads(4)
np.set_printoptions(precision=3, suppress=True)

import pandas as pd #library for data analysis
import matplotlib.pyplot as plt #library for creating static, animated, and interactive visualizations
import cv2 #Manipulate images and videos,
from PIL import Image # Python Imaging Library adds image processing capabilities to your Python interpreter. 
import math #mathematical functions
import requests #library for making HTTP requests

import faiss



2.18.0
Num GPUs Available:  0


In [3]:
model = tf.keras.models.load_model('model.keras')


In [13]:


embedding_model = tf.keras.Model(
    inputs=model.input,
    outputs=model.get_layer('embedding').output
)
from io import BytesIO

TCG_PATH = "../data/tcg/"

TCG_POKEMON_API_KEY = os.getenv("TCG_POKEMON_API_KEY")


embeddings_db = {}
card_ids = []

for json_key in os.listdir(TCG_PATH):
    data = json.load(open( f"{TCG_PATH}{json_key}", "r"))
    if data["id"].startswith("base1"):
        response = requests.get(data["images"]["small"],headers={ 'Accept': 'image/jpg'})
        img = Image.open(BytesIO(response.content))
        if img is not None:
            img = img.resize((224, 224))
            img = np.expand_dims(img, axis=0)  # Ajouter une dimension batch
            embedding = embedding_model.predict(img, verbose=0)[0]
            # Normaliser l'embedding
            embedding = embedding / np.linalg.norm(embedding)
        
            embeddings_db[data["id"]] = embedding
            card_ids.append(data["id"])

            print(f"Embedding: {embedding}")
        
np.save("embeddings_db.npy", embeddings_db)
np.save("card_ids.npy", card_ids)



Embedding: [0.134 0.    0.    0.032 0.    0.    0.    0.    0.064 0.015 0.196 0.
 0.    0.    0.    0.    0.    0.    0.004 0.062 0.121 0.016 0.077 0.035
 0.019 0.    0.097 0.153 0.146 0.019 0.    0.    0.    0.055 0.    0.
 0.012 0.068 0.    0.084 0.    0.064 0.045 0.    0.    0.    0.    0.032
 0.064 0.013 0.147 0.    0.075 0.    0.052 0.    0.061 0.112 0.02  0.061
 0.221 0.    0.    0.    0.093 0.    0.    0.    0.    0.    0.066 0.
 0.    0.    0.    0.    0.104 0.156 0.011 0.127 0.19  0.014 0.    0.
 0.    0.    0.063 0.078 0.    0.013 0.    0.    0.062 0.    0.    0.03
 0.    0.064 0.131 0.    0.    0.    0.127 0.1   0.    0.    0.109 0.
 0.017 0.    0.    0.034 0.    0.    0.032 0.008 0.047 0.019 0.043 0.
 0.    0.    0.069 0.095 0.195 0.    0.002 0.    0.    0.023 0.011 0.031
 0.062 0.085 0.    0.023 0.037 0.    0.    0.    0.008 0.    0.253 0.
 0.133 0.046 0.096 0.151 0.    0.    0.111 0.049 0.    0.122 0.    0.
 0.    0.075 0.    0.    0.    0.053 0.175 0.    0.    0.    0.  

In [15]:

# Charger les embeddings et les card_ids
embeddings_db = np.load("embeddings_db.npy", allow_pickle=True).item()
card_ids = np.load("card_ids.npy", allow_pickle=True)

# Convertir les embeddings en tableau numpy
embeddings = np.array(list(embeddings_db.values()))

# Créer un index FAISS
dimension = embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)  # Utilise le produit scalaire (équivalent à la similarité cosinus après normalisation)
index.add(embeddings)

# Sauvegarder l'index FAISS
faiss.write_index(index, "embeddings_index.faiss")

def find_similar_cards(query_img, index, card_ids, embedding_model, k=3):
    # Extraire l'embedding de la nouvelle carte
    query_img = np.expand_dims(query_img, axis=0)
    query_embedding = embedding_model.predict(query_img, verbose=0)[0]
    # Normaliser l'embedding
    query_embedding = query_embedding / np.linalg.norm(query_embedding)

    # Rechercher les k plus proches voisins
    distances, indices = index.search(np.expand_dims(query_embedding, axis=0), k)

    # Récupérer les card_ids correspondants
    similar_cards = [(card_ids[i], distances[0][j]) for j, i in enumerate(indices[0])]
    return similar_cards



# Exemple d'utilisation
query_img = Image.open("base1-7_1762430565.jpg")
similar_cards = find_similar_cards(query_img, index, card_ids, embedding_model, k=3)
print("Cartes similaires :", similar_cards)

Cartes similaires : [(np.str_('base1-65'), np.float32(0.8643346)), (np.str_('base1-102'), np.float32(0.8412103)), (np.str_('base1-101'), np.float32(0.8359425))]
